#### Build a Simple LLM Application with LCEL

In this quickstart, I did how to build a sumple LLM Application with Langchain. This application will translate text from english into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be build with just some prompting and an LLM call.


- Using langchain models
- Using PromptTemplates and OutputParsers
- Using LangChain Expression Langchain(LCEL) to chain components together
- Debugging and tracing application using LangSmith
- Deploying application with LangServe

In [1]:
## Open AI API Keys and Open Source Models -- Llama3, Gemma2, mistral -- Groq
## https://groq.com/

import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

groq_api_key = os.getenv("GROQ_API_KEY")


In [2]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

model = ChatGroq(model="llama-3.1-8b-instant", 
                 groq_api_key=groq_api_key)
model

c:\Users\amank\.conda\envs\langchain_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001C60B893FD0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001C60B8BC070>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following into French"),
    HumanMessage(content="Hello, How are you?")
]

res = model.invoke(messages)
res

AIMessage(content='Bonjour, comment allez-vous ?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 46, 'total_tokens': 54, 'completion_time': 0.017903225, 'completion_tokens_details': None, 'prompt_time': 0.00217669, 'prompt_tokens_details': None, 'queue_time': 0.05012429, 'total_time': 0.020079915}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bb792-0247-7d82-8d3c-e0e8d7eb85cc-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 46, 'output_tokens': 8, 'total_tokens': 54})

In [4]:
res.content

'Bonjour, comment allez-vous ?'

In [5]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parser.invoke(res)



'Bonjour, comment allez-vous ?'

In [6]:
## Using LCEL - Chain the components
chain = model | parser
chain.invoke(messages)

'Bonjour, comment allez-vous ?'

##### Another Way - Prompt Templates

In [7]:
from langchain_core.prompts import ChatPromptTemplate

generic_template = "Translate the following into {language}:"

prompt = ChatPromptTemplate.from_messages(
    [("system", generic_template),
     ("user", "{text}")
    ]
)



In [8]:
result = prompt.invoke(
    {"language" : "French",
     "text": "Hello"}
)

In [9]:
result.to_messages()

[SystemMessage(content='Translate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [10]:
## Chaining together components with LCEL
chain = prompt | model | parser
chain.invoke(
    {
        "language": "French",
        "text": "Hello"
    }
)

'Bonjour'